In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
}

base_url = 'https://www.wondertech.co.ls'
productlinks = []

In [3]:
# Define the Product class
class Product:
    def __init__(self, name=None, ratings=None, reviews=None, price=None):
        self.name = name
        self.ratings = ratings
        self.reviews = reviews
        self.price = price

    def to_dict(self):
        return {
            'name': self.name,
            'ratings': self.ratings,
            'reviews': self.reviews,
            'price': self.price
        }

    def __repr__(self):
        return f"Product(Name: {self.name}, Ratings: {self.ratings}, Reviews: {self.reviews}, Price: {self.price})"

products = []

In [4]:
# Loop through pages
for x in range(1, 4):
    url = f'{base_url}/shop?page={x}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    productlist = soup.find_all('div', class_='col-lg-4 col-md-6 col-sm-6 pb-1')

    # Find and store unique product links
    for item in productlist:
        for link in item.find_all("a", href=True):
            href = link['href']
            if href not in productlinks and href != 'https://#?':
                # Check if it's a relative URL and complete it
                full_link = href if href.startswith('http') else base_url + href
                productlinks.append(full_link)


In [5]:
# Iterate over product links and fetch product details
for link in productlinks:
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'lxml')
    
    # Create a dictionary to hold the product attributes
    product_data = {}
    
    # Find product name (h3 tag)
    name_tag = soup.find('h3')
    if name_tag:
        product_data['name'] = name_tag.text.strip()
    
    # Find the review rating (based on stars)
    stars = soup.find_all('small', class_='fas fa-star')
    half_stars = soup.find_all('small', class_='fas fa-star-half-alt')
    if stars or half_stars:
        ratings = len(stars) + (0.5 * len(half_stars))
        product_data['ratings'] = ratings
    
    # Find the number of reviews
    reviews_tag = soup.find('small', class_='pt-1')
    if reviews_tag:
        product_data['reviews'] = reviews_tag.text.strip()
    
    # Find the price
    price_tag = soup.find('h3', class_='font-weight-semi-bold mb-4')
    if price_tag:
        product_data['price'] = price_tag.text.strip()
    
    # Only create a Product object if we have at least one valid attribute
    if product_data:
        product = Product(**product_data)
        products.append(product)

In [6]:
# Convert the list of products to a list of dictionaries
product_dicts = [product.to_dict() for product in products]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(product_dicts)

In [8]:
# Output the DataFrame
print(df)

# To save to a CSV file:
# df.to_csv('products.csv', index=False)

                                  name  ratings       reviews       price
0       DS-2CE16DOT-IRF 2.8 2MP BULLET      3.5  (99 Reviews)     M436.00
1        PD Power 18CH PSU 12V 15A PTC      3.5  (99 Reviews)     M859.00
2       HIKVISION 8CH PSU 12VDC SURVEI      3.5  (99 Reviews)     M221.00
3                     PD POWER 9CH 12V      3.5  (99 Reviews)     M708.00
4        DS-2FA1208-C16 16CH HIK POWER      3.5  (99 Reviews)     M290.00
5       PSU 12VDC 4CH SURVEILLANCE HIK      3.5  (99 Reviews)     M209.00
6               16-ch 1U 16 PoE 4K NVR      3.5  (99 Reviews)  M4, 113.00
7                     4-ch Mini 1U NVR      3.5  (99 Reviews)     M929.00
8                      16-ch 1U 4K NVR      3.5  (99 Reviews)  M2, 063.00
9               8-ch Mini 1U 8 PoE NVR      3.5  (99 Reviews)  M2, 071.00
10   16-ch 1.5U 16 POE AcuSense 4K NVR      3.5  (99 Reviews)  M8, 220.00
11          16-ch 1.5U AcuSense 4K NVR      3.5  (99 Reviews)  M4, 958.00
12                8-ch 1U 8 PoE 4K NVR

In [9]:
# Sample list of random reviews
review_texts = [
    "Great product, fast delivery!",
    "Satisfactory but could be better.",
    "Exceeded my expectations, very happy.",
    "Quality is good, but delivery was slow.",
    "Not worth the price, quite disappointed.",
    "Works as expected, nothing special.",
    "Amazing quality, highly recommend!",
    "Terrible experience, would not buy again.",
    "Solid product, will purchase again.",
    "Good value for money."
]

In [10]:
# Generate random ratings and reviews for 21 products and 10 customers
num_products = 21
num_customers = 10
# product_names = [f'Product {i}' for i in range(1, num_products + 1)]
customer_ids = [f'Customer {i}' for i in range(1, num_customers + 1)]

In [11]:
# List to store the product review data
reviews_data = []

# Loop through each product and customer to generate random ratings and reviews
for product in product_dicts:
    for customer in customer_ids:
        rating = random.randint(1, 5)  # Random rating between 1 and 5
        review = random.choice(review_texts)  # Random review from the list
        reviews_data.append([product, customer, rating, review])

In [12]:
# Convert the data into a DataFrame for easy manipulation and analysis
df2 = pd.DataFrame(reviews_data, columns=['Product', 'Customer', 'Rating', 'Review'])

# Output the DataFrame
print(df2)

# Save to a CSV file
# df.to_csv('randomized_reviews.csv', index=False)

                                               Product     Customer  Rating  \
0    {'name': 'DS-2CE16DOT-IRF 2.8 2MP BULLET', 'ra...   Customer 1       5   
1    {'name': 'DS-2CE16DOT-IRF 2.8 2MP BULLET', 'ra...   Customer 2       5   
2    {'name': 'DS-2CE16DOT-IRF 2.8 2MP BULLET', 'ra...   Customer 3       3   
3    {'name': 'DS-2CE16DOT-IRF 2.8 2MP BULLET', 'ra...   Customer 4       1   
4    {'name': 'DS-2CE16DOT-IRF 2.8 2MP BULLET', 'ra...   Customer 5       3   
..                                                 ...          ...     ...   
205  {'name': '4-ch 1080p Mini 1U H.265 DVR', 'rati...   Customer 6       4   
206  {'name': '4-ch 1080p Mini 1U H.265 DVR', 'rati...   Customer 7       5   
207  {'name': '4-ch 1080p Mini 1U H.265 DVR', 'rati...   Customer 8       2   
208  {'name': '4-ch 1080p Mini 1U H.265 DVR', 'rati...   Customer 9       2   
209  {'name': '4-ch 1080p Mini 1U H.265 DVR', 'rati...  Customer 10       2   

                                        Review  
0 